In [1]:
!ls

20180527_Test_Cohorts.ipynb  Online Retail.xlsx


In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt

In [ ]:
online = pd.read_excel('Online Retail.xlsx')

In [ ]:
online[:5]

In [ ]:
online.dtypes

### Convert unique customer identifier into string
It will be easier to use for later aggregation

In [ ]:
online['CustomerID'] = online['CustomerID'].astype(str)

In [ ]:
len(set(online['CustomerID']))

### Create an "InvoiceMonth" column
This column will be later used for defining the acquisition month for each customer

In [ ]:
online['InvoiceMonth'] = online['InvoiceDate'].apply(lambda x: dt.datetime(x.year, x.month, 1))

### Assign acquisition month cohort per customer
We're going to use **transform** function on the dataframe to calculate the first month of each customer's purchase that will define their month of acquisition.

In [ ]:
online['CohortMonth'] = online.groupby('CustomerID')['InvoiceMonth'].transform('min')

### Calculate cohort index - ***nth*** month offset
Cohort index will allow us to normalize the metrics and compare them for each monthly cohort. This way we can assess the performance of each monthly cohort for a given ***nth*** month after customer's acquisition.

What we do is take a difference between the dates, and add plus one, in order to have one based indexing which is easier to interpret in a business context.

In [ ]:
online['CohortIndex'] = (
    online['InvoiceMonth'].apply(lambda x: x.year) - online['CohortMonth'].apply(lambda x: x.year))*12 + \
    (online['InvoiceMonth'].apply(lambda x: x.month) - online['CohortMonth'].apply(lambda x: x.month)) + 1

### Creating a string cohort month value
We will use this column to create a retention heatmap, and it will look much more beautiful in a short year-month format, than a full timestamp.

In [ ]:
online['CohortMonth'] = online['CohortMonth'].apply(lambda x: x.strftime('%Y-%m'))

In [ ]:
online.head()

### Create cohorts dataset with raw metrics
This is a fundamental step where metrics are defined. In this case we are creating three metrics: customer count, average price, and average quantity. We are going to use only the first one - customer count - to measure retention rate.

In [ ]:
grouping = online.groupby(['CohortMonth', 'CohortIndex'])

cohort_data = grouping.agg({'CustomerID': pd.Series.nunique,
                           'UnitPrice': np.mean,
                           'Quantity': np.mean})

cohort_data.rename(columns = {'CustomerID': 'CustomerCount',
                           'UnitPrice': 'AveragePrice',
                           'Quantity': 'AverageQuantity'}, inplace = True)

### Create a pivot table with active purchasers from each cohort
We will create a pivot table with ***CohortMonth*** in rows, and ***CohortIndex*** in columns, with ***CustomerCount*** as the values. 

In [ ]:
cohort_data.reset_index(level=['CohortMonth', 'CohortIndex'], inplace=True)
cohort_counts = cohort_data.pivot(index='CohortMonth', columns='CohortIndex', values='CustomerCount')

In [ ]:
cohort_counts

### Calculate retention rates
First, we're going to extract cohort size on the first month of their tenure with the company. Next, we will divide the number of active purchasers from each cohort by the size of that cohort. The result is a retention metric per each monthly cohort. 

In [ ]:
cohort_sizes = cohort_counts[1].values
retention = cohort_counts.divide(cohort_sizes, axis=0)

In [ ]:
retention

### Plot retention heatmap
We will use **Seaborn** library which we have imported earlier to build an easy-to-read heatmap chart.

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 8))
plt.title('Retention rates')
sns.heatmap(retention, annot=True, fmt='.0%', vmin=0.0, vmax=0.5, cmap='BuGn')
plt.show()